In [1]:
from DWCmod.DWC_models import KimKim2011 as model
from SALib.sample import saltelli
from SALib.analyze import sobol
import numpy as np
import pandas as pd

## Define functions

In [2]:
def res_dict_to_df(d, problem):
    """ Transform result dictionary from SALib to Pandas Dataframe"""
    ind_names = list(d.keys())[:4]
    data = list(d.values())[:4]
    col_names = problem["names"]
    df = pd.DataFrame(data=data, index=ind_names, columns=col_names)
    return df

def get_q(params):
    q = model(**params)[0]
    return q

def get_modelresults(params):
    """ calculate alpha from model output """
    if "deltaT_sub" in params:
        raise BaseException("deltaT_sub cannot be varied when calculating alpha")
    q = []
    bond = []
    deltaT_subs = [1, 10]
    for deltaT_sub in deltaT_subs:
        q.append(model(deltaT_sub=deltaT_sub, **params)[0])
        bond.append(model(deltaT_sub=deltaT_sub, **params)[9]["Bo"])
    grad = np.gradient(q, deltaT_subs)
    alpha = grad.mean()
    bond = sum(bond) / len(bond)
    return alpha, bond

def sensitivity(input_ranges, n_samples=1000):
    """ run the actual sensitivity analysis using sobol/satelli method"""
    # create martrix with input parameters
    problem = {
        "num_vars": len(input_ranges),
        "names": list(input_ranges.keys()),
        "bounds": list(input_ranges.values())
    }

    param_samples = saltelli.sample(problem, n_samples)
    print("Number of Samples, number of input parameters:", param_samples.shape)
    # calculate results vector
    Y = np.zeros([param_samples.shape[0]])
    bond = np.zeros([param_samples.shape[0]])
    for i, X in enumerate(param_samples):
        params = {list(input_ranges.keys())[j] : X[j] for j in range(len(X))}
        modelresults = get_modelresults(params)
        Y[i] = modelresults[0]
        bond[i] = modelresults[1]
    # analyse input matrix and results vector
    d_results = sobol.analyze(problem, Y)
    df = res_dict_to_df(d_results, problem)
    df = df.sort_values("S1", axis=1, ascending=False)
    return param_samples, Y, df, bond

def save_results(scen_name, df, df_samples):
    """ saves results as .pkl and .csv files """
    df.to_pickle("data//pickle//" + scen_name + "_indices.pkl")
    df_samples.to_pickle("data//pickle//" + scen_name + "_samples.pkl")
    df.to_csv("data//" + scen_name + "_indices.csv")
    df_samples.to_csv("data//" + scen_name + "_samples.csv")

# Sensitivity analysis
## Scenario 1
* large input parameter space based on physical plausibility
* no coating

In [3]:
scenario = {"p_steam": [50, 5000],                  # pressure in mbar
            "Theta": [5, 175],                      # contact angle in deg
            "CAH": [1, 90],                         # contact angle hysteresis in deg
            "N_s": [1, 1e6]                         # number of Nucleation sites in 10^9 1/m² 
               }

scen_name = "s1"

param_samples, alphas, df, bond = sensitivity(scenario)

# combine input parameters and results into one dataframe
df_samples = pd.DataFrame(data=param_samples, columns=scenario.keys())
df_samples["alpha"] = alphas
df_samples["bond"] = bond

save_results(scen_name, df, df_samples)

Number of Samples, number of input parameters: (10000, 4)


C:\Users\Sablowski\Documents\Python Scripts\TroKo\DWCmod\DWC_models.py:129: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  q_N, q_N_interr = integrate.quad(Q_drop_N, r_e, r_upper)


## Scenario 2
* large input parameter space based on physical plausibility
* consider coating

In [4]:
scenario = {"p_steam": [50, 5000],                  # pressure in mbar
            "Theta": [5, 175],                      # contact angle in deg
            "CAH": [1, 90],                         # contact angle hysteresis in deg
            "k_coat": [0.20, 10],                   # thermal conductivity of the coating in W/(mK)
            "delta_coat": [0.1e-6, 50e-6],          # thickness of the coating in m
            "N_s": [1, 1e6]                         # number of Nucleation sites in 10^9 1/m² 
               }

scen_name = "s2"

param_samples, alphas, df, bond = sensitivity(scenario)

# combine input parameters and results into one dataframe
df_samples = pd.DataFrame(data=param_samples, columns=scenario.keys())
df_samples["alpha"] = alphas
df_samples["bond"] = bond

save_results(scen_name, df, df_samples)

Number of Samples, number of input parameters: (14000, 6)


## Scenario 3
* narrow input parameter space based on rough estimates of measurement uncertainty
* no coating

In [5]:
scenario = {"p_steam": [110, 130],                  # pressure in mbar
            "Theta": [83, 93],                      # contact angle in deg
            "CAH": [24, 44],                        # contact angle hysteresis in deg
            "N_s": [1 , 1e6]                        # number of Nucleation sites in 10^9 1/m² 
               }

scen_name = "s3"

param_samples, alphas, df, bond = sensitivity(scenario)

# combine input parameters and results into one dataframe
df_samples = pd.DataFrame(data=param_samples, columns=scenario.keys())
df_samples["alpha"] = alphas
df_samples["bond"] = bond

save_results(scen_name, df, df_samples)

Number of Samples, number of input parameters: (10000, 4)


## Scenario 4
* narrow input parameter space based on rough estimates of measurement uncertainty
* consider good coating (thin, high thermal conductivity)

In [6]:
scenario = {"p_steam": [110, 130],                  # pressure in mbar
            "Theta": [83, 93],                      # contact angle in deg
            "CAH": [24, 44],                        # contact angle hysteresis in deg
            "k_coat": [5, 10],                      # thermal conductivity of the coating in W/(mK)
            "delta_coat": [0.1e-6, 1e-6],           # thickness of the coating in m
            "N_s": [1, 1e6]                         # number of Nucleation sites in 10^9 1/m² 
               }

scen_name = "s4"

param_samples, alphas, df, bond = sensitivity(scenario)

# combine input parameters and results into one dataframe
df_samples = pd.DataFrame(data=param_samples, columns=scenario.keys())
df_samples["alpha"] = alphas
df_samples["bond"] = bond

save_results(scen_name, df, df_samples)

Number of Samples, number of input parameters: (14000, 6)
